In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from multiprocessing import Pool

import numpy as np
import pandas as pd

import kraft

In [ ]:
directory_path = "/home/kwat/garden/data/cancer_cell_line/gene_set/"

kraft.path.path(directory_path)

In [ ]:
gene_x_sample = pd.read_csv(
    "{}/../rna/gene_x_cell_line_x_rna.tsv".format(directory_path), sep="\t", index_col=0
)

gene_x_sample = gene_x_sample.loc[gene_x_sample.index & kraft.get_gene_symbol(), :]

In [ ]:
gene_set_to_genes = kraft.read_gmts(
    (
        "../../gene_set/h.all.v7.1.symbols.gmt",
        "../../gene_set/c1.all.v7.1.symbols.gmt",
        "../../gene_set/c2.all.v7.1.symbols.gmt",
        "../../gene_set/c3.all.v7.1.symbols.gmt",
        "../../gene_set/c5.all.v7.1.symbols.gmt",
        "../../gene_set/c6.all.v7.1.symbols.gmt",
        "../../gene_set/c7.all.v7.1.symbols.gmt",
        "../../gene_set/ipa.gmt",
    )
)

In [ ]:
def function(scores, set_to_elements):

    scores = pd.Series(
        kraft.array.normalize(scores.to_numpy(), "-0-"),
        index=scores.index,
        name=scores.name,
    ).sort_values()

    return tuple(
        kraft.score_set(scores, elements, method=method, sort_scores=False, plot=False)
        for elements in set_to_elements.values()
    )

In [ ]:
method = "2020"

In [ ]:
pool = Pool(processes=10)

gene_set_x_sample = pd.DataFrame(
    np.asarray(
        pool.starmap(
            function,
            ((scores, gene_set_to_genes) for _, scores in gene_x_sample.items()),
        )
    ).T,
    index=gene_set_to_genes.keys(),
    columns=gene_x_sample.columns,
)

gene_set_x_sample.to_csv(
    "{}/gene_set_x_sample_x_{}.tsv".format(directory_path, method), sep="\t"
)

gene_set_x_sample